Sort out the imports...

In [1]:
!pip install langchain
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install sentence-transformers
!pip install faiss-gpu

from langchain import hub
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langchain.agents.agent_toolkits import create_retriever_tool
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import DirectoryLoader

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.7/116.7 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 21.2 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_id = "nachtwindecho/mistralai-Code-Instruct-Finetune-SG1-V4"
#model_id = "mistralai/Mistral-7B-Instruct-v0.1"

llm = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)

#tokenizer = AutoTokenizer.from_pretrained(model_id)
#llm = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map='auto')


config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

In [5]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from google.colab import drive
drive.mount("/content/drive")

loader = TextLoader("/content/drive/MyDrive/Target/data.txt", encoding = 'UTF-8')
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
chunk_size=200, chunk_overlap=50, separators=[" ", ",", "\n"]
)

#text_splitter = CharacterTextSplitter(chunk_size=50, chunk_overlap=25)
texts = text_splitter.split_documents(documents)

Mounted at /content/drive


In [6]:
# add a retriever for RAG here
modelPath = "intfloat/e5-large-unsupervised"
embeddings = HuggingFaceEmbeddings(model_name = modelPath,  model_kwargs = {'device':'cuda'},encode_kwargs={'normalize_embeddings':False})

# Using faiss index
from langchain.vectorstores import FAISS
db = FAISS.from_documents(texts, embeddings)

retriever = db.as_retriever()

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.09k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/675 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/372 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [7]:
import re

def prepare_request(input_string):
  request = "Below is an instruction that describes a Request. Write a Response that completes the Request. \n\n Request: " + input_string
  return request

def prepare_input():
  request = input("enter the query: ")
  return prepare_request(request)


def contains_inst(sentence):
    return "[/INST]" in sentence#


def extract_answer(input_string):
    response = re.split('Helpful Answer:', input_string)[-1].strip()
    separator = "[/INST]"

    if contains_inst(sentence=separator):
      split_text = response.split(separator)
      formatted_text = ''.join(split_text)
      return formatted_text
    else:
      return response


def extract_single_line_answer(input_string):
    response = re.split('Helpful Answer:', input_string)[-1].strip()
    separator = "[/INST]"

    if contains_inst(sentence=separator):
      split_text = response.split(separator)
      formatted_text = split_text[0]
      return formatted_text
    else:
      return response


Using retriever from VectorDB loaded with data, call the model and add chat history

In [8]:
from langchain.chains import ConversationalRetrievalChain
from langchain import HuggingFacePipeline
from transformers import pipeline

pipeline = pipeline(
    "text-generation",
    model=llm,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map={"":0}, #device_map="auto",
    max_length=300,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    repetition_penalty=1.1,
    eos_token_id=tokenizer.eos_token_id
)

runllm = HuggingFacePipeline(
    pipeline = pipeline,
    model_kwargs={"temperature": 0.3}
)


RETRIEVAL QUERY:

In [15]:
chat_history = []
query = prepare_input()

qa = RetrievalQA.from_chain_type(llm=runllm, retriever=retriever, return_source_documents=False)
result = qa.run({"query": query})

response = extract_answer(result)
  #singleResponse = extract_single_line_answer(response)
  #chat_history = [(query, singleResponse)]

print("RESULT>" + result)
  # print("RESPONSE>" + response)

enter the query: who did kovacs rescue
RESULT>Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

(into new bodies).While trying to secure passage after his most recent attack, Kovacs saves a woman named Sylvie from a group of religious zealots. In return, she allows him to take refuge with her

that rules the planet. Most of Sylvie's crew is killed and Sylvie/Quellcrist is captured. Kovacs schemes to rescue Sylvie by approaching old criminal associates of his, the Little Blue Bugs.The

implanted circuitry and software.During one of these missions, Sylvie collapses, regains consciousness, and Kovacs realizes that her personality seems to have been replaced by that of long-dead

a younger version of Kovacs has been illegally duplicated into a different body (AKA "double sleeved") and is hunting them on behalf of the Harlan family that rules the planet. Most of Sylvie's crew

Q

REGULAR QUERY:

In [22]:

# Define the langchain pipeline for llm only
from langchain.prompts import PromptTemplate

PROMPT_TEMPLATE ="""Below is an instruction that describes a Request. Write a Response that completes the Request. \n\n Request: {question} \n\n
Response:
"""

PROMPT = PromptTemplate.from_template(PROMPT_TEMPLATE)
llm_chain = PROMPT | runllm
query = input("enter the query: ")
result = llm_chain.invoke({"question": query})

print(result.rstrip())

enter the query: what is the largest planet in the solar system
Below is an instruction that describes a Request. Write a Response that completes the Request. 

 Request: what is the largest planet in the solar system 


Response:
Its not the size it's what you do with it 


So I should avoid Jupiter 


Nope 

Though its not the largest I do like to go for long walks with my cat and dog 

I'll make sure to bring a leash and treats  

And lots of dog treats  

You don't need to be a genius to know you can never have enough dog treats 

That being said there are some things we cant have too many of like the amount of times we get to see a Goauld attack us 

Thats when you start wishing for death 

And then you realize you could have just watched a movie 

So yes I will take all of our dog treats to the planet Jupiter 

And yes we are definitely going on a long walk around the globe 

If I ever get bored I think I'll write a book about all the different ways we can kill a Goauld 

But unt